In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
torch.manual_seed(42)

In [ ]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/fashion-mnist_train.csv'

In [ ]:
# Create a 4x4 grid of images
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle("First 16 Images", fontsize=16)

# Plot the first 16 images from the dataset
for i, ax in enumerate(axes.flat):
    img = df.iloc[i, 1:].values.reshape(28, 28)  # Reshape to 28x28
    ax.imshow(img)  # Display in grayscale
    ax.axis('off')  # Remove axis for a cleaner look
    ax.set_title(f"Label: {df.iloc[i, 0]}")  # Show the label

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the title
plt.show()


In [ ]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [ ]:
train_dataset = CustomDataset(X_train, Y_train)

In [ ]:
train_dataset[0]

In [ ]:
test_dataset = CustomDataset(X_test, Y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, pin_memory = True)

In [ ]:
class myNN(nn.Module):
  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer):
    super().__init__()
    layers = []
    for i in range(num_hidden_layers):
        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(nn.BatchNorm1d(neurons_per_layer))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(p = 0.3))
        input_dim = neurons_per_layer
    layers.append(nn.Linear(input_dim, output_dim))
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)


In [ ]:
def objective(trial):

  #hyperparameters to be tuned
  num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
  neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step = 8)

  #model init
  input_dim = 784
  output_dim = 10

  model = myNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer)
  model = model.to(device)

  #params init
  epochs = 100
  lr = 1e-1

  #optimiser selection
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr= lr, weight_decay = 1e-4)

  #training loop
  for epoch in range(epochs):
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        y_pred = model(features)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  #evaluation
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
      for batch_features, batch_labels in test_loader:
          batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
          output = model(batch_features)
          _, predicted = torch.max(output, 1)
          correct = correct + (predicted == batch_labels).sum().item()
          total += batch_labels.shape[0]
      accuracy = correct / total

  return accuracy


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 29.0 MB/s eta 0:00:00


In [ ]:
import optuna
study = optuna.create_study(direction = 'maximize')


[I 2025-12-18 11:21:46,923] Trial 0 finished with value: 0.8565 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 32}. Best is trial 0 with value: 0.8565.
[I 2025-12-18 11:29:03,887] Trial 1 finished with value: 0.8868333333333334 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 80}. Best is trial 1 with value: 0.8868333333333334.
[I 2025-12-18 11:33:14,200] Trial 2 finished with value: 0.847 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 16}. Best is trial 1 with value: 0.8868333333333334.
[I 2025-12-18 11:40:26,040] Trial 3 finished with value: 0.8564166666666667 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 32}. Best is trial 1 with value: 0.8868333333333334.
[I 2025-12-18 11:46:37,843] Trial 4 finished with value: 0.8915833333333333 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 112}. Best is trial 4 with value: 0.8915833333333333.
[I 2025-12-18 11:54:46,977] Trial 5 finished with value: 0.8743333333333333 and p